# Selenium으로 크롤링
### 아동보호시설
* http://www.safe182.go.kr/lom/listLcInfoMap.do

In [11]:
# intro
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

driver = webdriver.Chrome()
driver.get("http://www.safe182.go.kr/lom/listLcInfoMap.do")
driver.implicitly_wait(5)

driver.find_element_by_css_selector('#findChoice1').click()
driver.find_element_by_css_selector('#findChoice3').click()
driver.implicitly_wait(5)
driver.find_element_by_css_selector('.margin_bm10 .tabLeft2').click()

In [12]:
html = BeautifulSoup(driver.page_source, "lxml")
address_list = []
title_list = []
latlan_x_list = []
latlan_y_list = []
words_list = []

In [13]:
# <address> 태그 모두 찾기.
def address():
    html = BeautifulSoup(driver.page_source, "lxml")
    global address_list
    address_list2 = []
    addressList = html.find_all("address")  # bs4.element.ResultSet 형식으로 반환
    for address in addressList:
        address_list2.extend(address)
    
    addres_list2 = address_list2[:10]
    address_list.extend(addres_list2)

In [14]:
# class이름이 title인 모든 태그의 값 찾기.
def title():
    html = BeautifulSoup(driver.page_source, "lxml")
#     global title_list
    titleList = html.select(".title")   # list로 반환.
    
    for title in titleList:
        title_list.extend(title.contents)

In [15]:
# link에서 href 속성값 가져오기.
def latlan():
    html = BeautifulSoup(driver.page_source, "lxml")
    select = "li a.title"
    links = html.select(select)
    
    for link in links:
        if link.has_attr("href") == True:
            href = link["href"]
            words_list = href.split(sep="'")
            latlan_x_list.extend([words_list[1]])
            latlan_y_list.extend([words_list[3]])

In [16]:
# 페이지 이동
def nextPage(i):
    page = int(i)+1
    if page % 5 != 1:
        pageNo = str(page)
        driver.find_element_by_link_text(pageNo).click()
    elif page % 5 == 1:
        driver.find_element_by_css_selector('strong ~ .display_inb > a').click()
    if page % 100 == 0:
        print(pageNo)

In [17]:
for i in range(1,18):
    address()
    driver.implicitly_wait(30)
    title()
    driver.implicitly_wait(30)
    latlan()
    driver.implicitly_wait(30)
    nextPage(i)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"18"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


In [26]:
len(address_list)

166

In [27]:
len(title_list)

161

In [34]:
len(latlan_y_list)

161

In [28]:
import pandas as pd

# s:이름, s1:주소, s2:좌표
s = pd.DataFrame(title_list, columns=['이름'])
s1 = pd.DataFrame(address_list, columns=['주소'])
s2 = pd.DataFrame(latlan_x_list, columns=['x좌표'])
s3 = pd.DataFrame(latlan_y_list, columns=['y좌표'])

In [29]:
s.tail()

,이름
156,홀트아동복지회인천상담소
157,홍성군사회복지관
158,후생학원
159,희락원
160,희망샘학교


In [30]:
s1.head()

,주소
0,강원도 강릉시 강변로 450-0
1,강원도 강릉시 경강로2267번길 19-0
2,강원 원주시 개운동 288-8 남양빌딩4층
3,전라남도 강진군 동성로 48-0
4,경기 의정부시 녹양동 318-5


In [33]:
s1.tail(6)

,주소
160,전라북도 고창군 학천로 267-16
161,(우) 04322 서울특별시 용산구 한강대로71길 37 아동·여성·장애인 경찰지원센터
162,"<span class=""colorD5"">|</span>"
163,
164,"<strong class=""color555"">국번없이</strong>"
165,"<strong class=""colorBlue"">182, 117</strong>"


In [35]:
s3.tail(6)

,y좌표
155,35.5565011
156,37.4615335
157,36.6018445
158,36.2982635
159,35.2931148
160,35.4129297


In [37]:
CareFacilities = ((s.join(s1)).join(s2)).join(s3)

In [39]:
CareFacilities.tail()

,이름,주소,x좌표,y좌표
156,홀트아동복지회인천상담소,인천 남구 주안동 161-5,126.6805092,37.4615335
157,홍성군사회복지관,충청남도 홍성군 홍성읍 오관리,126.6582497,36.6018445
158,후생학원,"대전광역시 서구 정림동,99-2",127.3672032,36.2982635
159,희락원,부산 금정구 노포동 964-1,129.0867196,35.2931148
160,희망샘학교,전라북도 고창군 학천로 267-16,126.5888365,35.4129297


# 정규식으로 주소가 서울인 곳만 추리기

In [40]:
import re

seoul_index = []
seoul_title = []
seoul_addr = []
seoul_x = []
seoul_y = []

addr = CareFacilities.주소

In [41]:
p = re.compile("(서울)")
for i,j in addr.items():
    if re.match(p, j) != None:
        seoul_index.append(i)
        seoul_addr.append(j)
seoul_index
c4 = []
for i in seoul_index:
    c4.append(childSafetyGuardHouse.loc[i])

In [42]:
final_df = pd.DataFrame(c4)
final_df.head()
len(final_df)

11

In [43]:
final_df.head()

,이름,주소,x좌표,y좌표
55,서울SOS어린이마을,"서울특별시 양천구 신월동,149-7",126.8260567,37.5305927
56,서울시립아동상담치료센터,서울 동대문구 장안동 329-1,127.0752636,37.5770123
60,성로원아기집,서울 동작구 노량진동 223-5,126.9460682,37.509397
74,시온원,서울특별시 동작구 상도1동,126.9498827,37.5027452
82,신월3동나눔의집,"서울특별시 양천구 신월동,195-18 4층",0.00137329,0.00137329


In [44]:
final_df.to_csv("서울아동보호시설.csv")